### Começamos realizando o load de algumas bibliotecas auxiliares que iremos utilizar

In [ ]:
from json import load as jsload
from tweepy import OAuthHandler, API
from warnings import catch_warnings, filterwarnings
from random import shuffle

### Agora importamos as funções do spacy que iremos utilizar e carregar o modelo pré treinado

In [ ]:
from spacy import load

nlp = load("pt_core_news_md")

### Coloquei as chaves de acesso a minha conta no Twitter em um arquivo separado :D

In [ ]:
with open("key.json") as file:
    keys = jsload(file)

In [ ]:
auth = OAuthHandler(
    consumer_key = keys["cunsumer_key"],
    consumer_secret = keys["consumer_secret"]
)

auth.set_access_token(
    key = keys["access_token"],
    secret = keys["access_token_secret"]
)
api = API(auth)

### Vamos baixar os dados do Twitter

In [ ]:
# Capturando dados do Twitter
tweets = {each._json["user"]["name"] : each._json["text"]  for each in api.search(q = "#bitcoin", lang = "pt", count = 200)}
tweets

### Vamos criar uma função que realize o filtro de palavras que desejamos trabalhar

In [ ]:
def word_filter(word, cut_stop = True):
    if word.is_stop and cut_stop:
        return False
    elif word.is_punct:
        return False
    elif word.suffix_ == "…":
        return False
    elif word.like_url:
        return False
    elif word.like_email:
        return False
    elif word.like_num:
        return False
    elif word.prefix_ == "@":
        return False
    elif word.text in [" ", "\n", "\n\n", "...", 'RT']:
        return False
    elif not word.text.isalnum():
        return False
    return True

### Realizando o pré processamento das palavras

In [ ]:
# Pré-processamento: Stop Words e Lemmatazing
processeded = []
# interando sobre cada tweet
for user, tweet in tweets.items():
    row = []
    for word in nlp(tweet): # este é o pipeline
        # filtrando as palavras
        if word_filter(word):
            # após selecionar as palavras, é adicionado o seu formato lematizado
            lemm = nlp.vocab[word.text]
            row.append(lemm.text)
    print(f"{user} : {row}")
    processeded.append(row)

In [ ]:
processeded = []
ner = []
adj = []
for each in tweets.values():
    doc = nlp(each)
    processeded.append([nlp.vocab[word.text].text for word in doc if word_filter(word)])
    ner.append([(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
ner